# Nature Conservancy Fish Classification - Bounding Box End-to-End Model

### Imports & environment

In [1]:
import os
from PIL import Image

from keras.callbacks import ModelCheckpoint
from keras.layers import GlobalAveragePooling2D, Activation, Input, Flatten
from keras.optimizers import Adam
from keras.models import Sequential
from keras.preprocessing.image import load_img, img_to_array

from vgg16bn import Vgg16BN as VggConv

from utils import * 
from models import Vgg16BN, Inception, Resnet50
from glob import iglob

ROOT_DIR = os.getcwd()
DATA_HOME_DIR = ROOT_DIR + '/data'
%matplotlib inline

Using TensorFlow backend.


In [2]:

ROOT_DIR

'/home/ipl/Downloads/0126/train/nc-fish-classification/nc_fish_classification'

### Config & Hyperparameters

In [3]:
# paths
data_path = DATA_HOME_DIR + '/cropped' 
split_train_path = data_path + '/train/'
valid_path = data_path + '/valid/'
test_path = DATA_HOME_DIR + '/test/'
saved_model_path = ROOT_DIR + '/models/bb_end_to_end/'
submission_path = ROOT_DIR + '/submissions/bb_end_to_end/'
fish_detector_path = ROOT_DIR + '/models/fish_detector_480x270/0.03-loss_2epoch_480x270_0.3-dropout_0.001-lr_vggbn.h5'
bb_regressor_path = 'models/bb_regressor/360x640/loss-1248.00_vgg16_bn.h5'

# data
batch_size = 16
im_size = (224, 224)  # (ht, wt); only 299x299 for inception
nb_split_train_samples = 2847
nb_valid_samples = 450
nb_test_samples = 1000
classes = ["ALB", "BET", "DOL", "LAG", "OTHER", "SHARK", "YFT"]
nb_classes = len(classes)

# model
nb_runs = 1
nb_epoch = 12
nb_aug = 1
dropout = 0.5
lr=0.001           
clip = 0.01
archs = ["vggbn"]

models = {
    "vggbn": Vgg16BN(size=im_size, n_classes=nb_classes, lr=lr,
                           batch_size=batch_size, dropout=dropout),
    "inception": Inception(size=(299, 299), n_classes=nb_classes,
                           lr=0.001, batch_size=batch_size),
    "resnet": Resnet50(size=im_size, n_classes=nb_classes, lr=lr,
                    batch_size=batch_size, dropout=dropout)
} 

### Build & Train Classifier

In [4]:
def train(parent_model, model_str):
    parent_model.build()    
    model_fn = saved_model_path + '{val_loss:.2f}-loss_{epoch}epoch_' + model_str
    ckpt = ModelCheckpoint(filepath=model_fn, monitor='val_loss',
                           save_best_only=True, save_weights_only=True)
    
    parent_model.fit_val(split_train_path, valid_path, nb_trn_samples=nb_split_train_samples, 
                         nb_val_samples=nb_valid_samples, nb_epoch=nb_epoch, callbacks=[ckpt], aug=nb_aug)

    model_path = max(iglob(saved_model_path + '*.h5'), key=os.path.getctime)
    return model_path

In [5]:
def train_all():    
    model_paths = {
        "vggbn": [],
        "inception": [],
        'resnet': [],
    }
    
    for run in range(nb_runs):
        print("Starting Training Run {0} of {1}...\n".format(run+1, nb_runs))
        aug_str = "aug" if nb_aug else "no-aug"
        
        for arch in archs:
            print("Training {} model...\n".format(arch))
            model = models[arch]
            model_str = "{0}x{1}_{2}_{3}lr_run{4}_{5}.h5".format(model.size[0], model.size[1], aug_str,
                                                                 model.lr, run, arch)
            model_path = train(model, model_str)
            model_paths[arch].append(model_path)
        
    print("Done.") 
    return model_paths
        
model_paths = train_all()

Starting Training Run 1 of 1...

Training vggbn model...



ResourceExhaustedError: OOM when allocating tensor with shape[25088,4096]
	 [[Node: random_uniform_13/RandomUniform = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=734155570, _device="/job:localhost/replica:0/task:0/gpu:0"](random_uniform_13/shape)]]

Caused by op 'random_uniform_13/RandomUniform', defined at:
  File "/home/ipl/anaconda2/envs/py3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ipl/anaconda2/envs/py3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ipl/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/ipl/anaconda2/envs/py3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/ipl/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/

### Generate Predictions

In [ ]:
def get_bb_pred(test_image):  
    
    # load conv model
    vgg_conv = VggConv((360, 640)).model
    vgg_conv.pop()
    vgg_conv.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])
    
    # build bb regressor & load weights
    input_shape = (512, 22, 40)  # shape of output of conv model
    
    model = Sequential()
    model.add(MaxPooling2D(input_shape=input_shape))
    model.add(BatchNormalization(axis=1))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(4))

    model.compile(Adam(lr=0.001), loss='mse')
    model.load_weights(bb_regressor_path)
    
    # extract conv features from conv model
    conv_test_feat = vgg_conv.predict(test_image)
    
    # predict bounding box coordinates
    bb_preds = model.predict(conv_test_feat)
    
    return bb_preds[0]

In [ ]:
def generate_preds(model):
    fish_detector = Vgg16BN(size=(270, 480), n_classes=2, lr=0.001,
                            batch_size=batch_size, dropout=dropout)
    fish_detector.build()
    fish_detector.model.load_weights(fish_detector_path)

    nofish_prob, filenames = fish_detector.test(test_path, nb_test_samples, aug=nb_aug)
    nofish_prob = nofish_prob[:, 1]
    
    predictions = []
    
    for ix, fn in enumerate(filenames):
        
        if ix % 100 == 0:
            print("Processing {0} of {1}".format(ix, len(filenames)))
        
        im_id = fn.split("/")[-1]
        im = Image.open(test_path + fn).resize((640, 360))
        test_x = np.array(im)
    
        # Nofish
        if nofish_prob[ix] > 0.5:
            alloc = (1. - nofish_prob[ix] / 7.)
            pred = np.array([alloc, alloc, alloc, alloc, nofish_prob[ix], alloc, alloc, alloc])
            predictions.append(pred)
            
        # Fish
        else:
            coords = get_bb_pred(test_x.reshape(1, 3, 360, 640))        
            ht = coords[0]
            wt = coords[1]
            x = coords[2]
            y = coords[3]
            mx_dim = max([wt, ht])

            cropped = im.crop((x, y, x + mx_dim, y + mx_dim)).resize((im_size[1], im_size[0]))
            test_x = np.array(cropped).reshape(1, 3, im_size[0], im_size[1])
            weight = -1. * (nofish_prob[ix] - 1.)
            
            pred = model.predict(test_x)
            pred = weight * pred
            pred = np.insert(pred, 4, nofish_prob[ix], axis=1)
            predictions.append(pred)
    
    return np.array(predictions), filenames

In [ ]:
def test(model_paths):   
                 
    print("----Predicting on {} model...".format(arch))
    parent = models["vggbn"]
    model = parent.build()
    model.load_weights(model_paths[arch][run])
    pred, filenames = generate_preds(model)
    
    predictions = []

    # weird, hacky reshaping
    for p in pred:
        if p.shape != (1, 8):
            p = p.reshape(1, 8)
        predictions.append(p)
        
    predictions = np.concatenate([p for p in predictions])
    return predictions, filenames

predictions, filenames = test(model_paths)

### Write Predictions to File

In [ ]:
def write_submission(predictions, filenames):
    preds = np.clip(predictions, clip, 1-clip)
    sub_fn = submission_path + '{0}epoch_{1}aug_{2}clip_{3}runs'.format(nb_epoch, nb_aug, clip, nb_runs)
    
    for arch in archs:
        sub_fn += "_{}".format(arch)

    with open(sub_fn + '.csv', 'w') as f:
        print("Writing Predictions to CSV...")
        f.write('image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT\n')
        for i, image_name in enumerate(filenames):
            pred = ['%.6f' % p for p in preds[i, :]]
            f.write('%s,%s\n' % (os.path.basename(image_name), ','.join(pred)))
        print("Done.")

write_submission(predicitions, filenames)